In [1]:
#Definindo finais de semana
mes = "09"
lista_fds = ['01','07','08','14','15','21','22','28','29']

# Funções e Pacotes

In [2]:
# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

#definindo quantidade de linhas e colunas visiveis
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

# Manipulação de Strings
import unidecode # => unidecode(string)
import re # Regex

# Repositorio - Diretorio
import os
import shutil

In [3]:
#######################-FUNÇÕES DE CONTAGEM
def count_freq(data_frame, coluna):
    freq = df[coluna].value_counts()
    perc = df[coluna].value_counts(normalize = True)*100
    tabela = pd.DataFrame({"Freq":freq, "Perc":perc})
    return display(tabela)
        
def tabela_cont(linha, coluna):
    a = df.groupby([linha, coluna]).size().unstack()
    for i in a.columns:
        a[i].fillna(0, inplace = True)
        a[i] = a[i].apply(lambda x: int(x))
    return a
    print('='*100)

def tabela_cont_percent(linha, coluna, opcao):
    result = tabela_cont(linha, coluna)
    if opcao == 'Linha':
        return round(result.div(result.sum(axis=1), axis=0) * 100, 2)
    elif opcao == 'Coluna':
        return round(result.div(result.sum(), axis=1) * 100, 2)
    else:
        return 'Opção invalida'

In [4]:
# Funções de salvar arquivo
def salvando_nome_arquivo(nome_arquivo, dataframe):
    if os.path.exists(nome_arquivo): #O nome_arquivo existe
        df1 = pd.read_excel(nome_arquivo)
        len_antigo = len(dataframe)
        
        dataframe = pd.concat([df1, dataframe]).reset_index(drop=True)
        dataframe = dataframe.drop_duplicates(subset=['chat'])
        
        print(len(df1), " += ", len_antigo, "=>", len(dataframe))
        
        dataframe.to_excel(nome_arquivo, index=False)
    
    else: #O nome_arquivo não existe
        dataframe.to_excel(nome_arquivo, index=False)
        print(len(dataframe)) 

In [5]:
def read_file(file_path):
    # 1. Ler o arquivo txt
    with open(file_path, "r", encoding="utf-8") as file:
        lines = file.readlines()
    
    # 2. Remover todas as linhas que contêm "Mensagem apagada"
    lines = [line for line in lines if "Mensagem apagada" not in line]
    
    # 3. Corrigir as mensagens que estão em várias linhas para que fiquem em uma única linha
    corrected_lines = []
    current_message = ""
    
    for line in lines:
        # Se a linha começa com o padrão de data e hora, ela é uma nova mensagem
        if re.match(r"^\[\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}\]", line):
            if current_message:  # Se há uma mensagem anterior, adicione-a à lista
                corrected_lines.append(current_message)
            current_message = line.strip()  # Comece uma nova mensagem
        else:
            current_message += " " + line.strip()  # Adicione o conteúdo da linha atual à mensagem em progresso
    
    # Adiciona a última mensagem, se houver
    if current_message:
        corrected_lines.append(current_message)
    
    # Salvando arquivo txt
    with open('processed.txt', "w", encoding="utf-8") as file:
        for line in corrected_lines:
            file.write(line + "\n")
    
    # Lendo arquivo txt e transformando em dataframe cada mensagem
    with open('processed.txt', 'r', encoding='utf-8') as file:
        content = file.read().splitlines()
    
    return content

In [6]:
def sample_prob(nivel):
    '''
    data = df_prob[df_prob['controle'] == nivel]
    tamanho = len(data)
    try:
        if tamanho == 0:
            data_exibir = str('0 itens do prob' + nivel)
        if tamanho == 1:
            data_exibir = data['mensagem'].sample(1)
        if tamanho == 2:
            data_exibir = data['mensagem'].sample(2)
        if tamanho > 5:
            data_exibir = data['mensagem'].sample(5)
    except:
        data_exibir = 'erro'
    '''
    return 'nada'

# =========================================

# Carregando base

In [7]:
df = pd.DataFrame({'chat': read_file('_chat.txt') })
#df = pd.read_excel('df_residuos.xlsx')[['chat']]; os.remove("df_residuos.xlsx")

df['controle'] = np.nan
df = df[['controle','chat']]
df_prob = pd.DataFrame(columns = ['controle','chat'])

print(len(df))
display(df.head())

705


,controle,chat
0,NaN,[20/09/2024 20:47:00] Letícia Fernandes Borges...
1,NaN,[20/09/2024 20:48:16] Karen Cavichini Rosseto:...
2,NaN,[20/09/2024 20:53:50] Gabriela Soares Piassi: ...
3,NaN,[20/09/2024 20:54:10] Débora Alves Aude: 20/09...
4,NaN,[20/09/2024 20:56:17] Izabela Cristina Almeida...


# Dia de envio da mensagem

In [8]:
dia = []
for i in range(0, len(df['chat'])):
    dia.append(df['chat'][i][1:11]) #pegando exatamente aonde parte da data esta
df['dia'] = dia
#print(sorted(df['dia'].unique()))
print("Quantidade de mensagens por dia:")
count_freq(df, 'dia')

Quantidade de mensagens por dia:


,Freq,Perc
dia,,
23/09/2024,227,32.198582
24/09/2024,197,27.943262
21/09/2024,182,25.815603
22/09/2024,66,9.361702
20/09/2024,33,4.680851


# Nome Anestesista

In [9]:
df['nome'] = np.nan
count = 0
for i in range(0, len(df['chat'])):
    loc = df['chat'][i].find(']')+1 #digito data
    loc2 = df['chat'][i][loc+1:].find(':')+1 #digito nome
    nome = df['chat'][i][loc+1:loc+loc2]
    nome = nome.upper().strip().replace("  ", " ")
    nome = unidecode.unidecode(nome)
    df['nome'][i] = nome
    count += 1
    #print(count, "->", unidecode.unidecode(chat[i][loc+1:loc+loc2].upper()))

print("Profissionais que mandaram mensagem:")
df['nome'] = df['nome'].apply(lambda x: x.replace('  ',' '))
sorted(df['nome'].unique())

Profissionais que mandaram mensagem:


['ALEXANDRE BERNARDINO',
 'ALEXSANDER TAKASHI HAMADA',
 'ALINE FRANCHINI GOULART LIMA',
 'AMANDA KIYOKO SOUZA KOSSAKA',
 'ANA JUNG DOCKO',
 'ANDRE AUGUSTO BELLINI',
 'ANDRE DA COSTA AGUIAR VENTURA',
 'ANDREIA LOUREIRO MORI',
 'ANNA BEATRIZ ARANHA TREVISAN',
 'ANTONIO ALVARO DIAS LIMA DA CUNHA NETO',
 'ARTHUR OLIVEIRA FACCHINI',
 'ARTHUR RODRIGUES TORRELIO',
 'AUGUSTO DE CASTRO SANTOS QUEIROZ',
 'BACKOFFICE PACIENTES',
 'BIANCA PRISCILA BUNIOTTI',
 'BRIAN SOSSAI PACHECO',
 'BRUNA DAIBERT DANESIN',
 'BRUNO FERRAZ DE CAMPOS BARONE',
 'BRUNO STORNIOLI TAGUCHI LEMOS',
 'CAIO TULIO DUARTE',
 'CAMILA RESENDE DE PAULA',
 'CARLOS AUGUSTO TREVISAN',
 'CARLOS EDUARDO CORREA GRILO',
 'CAROLINE FIRMINO LEMES DA SILVA',
 'CLAUDIO MUELLER KAKUDA',
 'DANIEL MASUZAKI WONG',
 'DANIEL RODRIGUES DE OLIVEIRA',
 'DANIELLEN LINS LOURENCO',
 'DEBORA ALVES AUDE',
 'DENIO LOPES DE ALMEIDA',
 'DIEGO DA CUNHA TOMAZ',
 'DIOGO CANDEO RODRIGUES CORDEIRO',
 'DIOGO HENRIQUE PEREIRA MATTOS',
 'EDUARDO ARBACHE BEZERRA',

# Mensagens escrevidas e enviadas

In [10]:
mensagem = []
for i in range(0, len(df)):
    digito_data = df['chat'][i].find(']') #aonde acaba a data
    digito_nome = df['chat'][i][digito_data+1:].find(':')
    mensagem_escrevida = df['chat'][i][digito_data+digito_nome+3:]
    mensagem.append(mensagem_escrevida)
    
df['mensagem'] = mensagem
df['mensagem'] = df['mensagem'].apply(lambda x: x.upper() )
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('-',' - '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('(',' ( '))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace(')',' ) '))

print('Mensagens Enviadas:')
df['mensagem'].sample(3)

Mensagens Enviadas:


161                 21/09 ANALIA  -  MANUELA SAÍDA 13:43
336    23/09 OSASCO  -  M TERESA ENTRADA 07:00  ( PÓS...
556                  24/09 ALPHA  -  MORAES ENTRADA 7:00
Name: mensagem, dtype: object

# =========================================

# Data do Plantão

In [11]:
df['data'] = np.nan

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
    else: #restante com nenhuma ou mais de uma data
        df['controle'][i] = 'DATA_DO_PLANTÃO'

#salvando
df_prob = df[df['controle'] == 'DATA_DO_PLANTÃO']
print("data probs:", len(df_prob))
sample_prob('DATA_DO_PLANTÃO')

df = df[df['controle'] != 'DATA_DO_PLANTÃO']
print("datas ok:", len(df)) 

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

count_freq(df, 'data')
#print("Dias mensionados nas mensagens:")
#print(sorted(df['data'].unique()))

data probs: 9
datas ok: 696


,Freq,Perc
data,,
23/09,222,31.896552
24/09,187,26.867816
21/09,170,24.425287
22/09,63,9.051724
20/09,49,7.040230
23/9,3,0.431034
22/08,1,0.143678
24/9,1,0.143678


# Unidades

In [12]:
df['unidade'] = ''
df['Q_unidades'] = ''

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    
    #Unidades escrita com a bunda
    lista_avi = ["CENTRAL","AVICENA","AVICENNA","AVICCENNA",'TATUAPE']
    for i_avi in lista_avi:
        mensagem = mensagem.replace(i_avi, 'AVICCENA')
        
    mensagem = mensagem.replace('Sta Isabel', 'SANTA')
    
    lista_unidades = re.findall(r'ALPHA|ANALIA|AVICCENA|ITAIM|JABA|MATERNIDADE|MORUMBI|OSASCO|VILLA|VILA|VNS|SANTA', mensagem)
    quantidade_unidades = len(lista_unidades)
    
    df['Q_unidades'][i] = quantidade_unidades
    
    if quantidade_unidades == 1: #capturando apenas com uma unidade
        df['unidade'][i] = lista_unidades[0].replace('  ',' ').upper().strip()
    elif quantidade_unidades > 1: #com mais de uma unidade
        df['unidade'][i] = lista_unidades
        df['controle'][i] = 'DESLOCADO'
    else: #sem nenhuma unidade
        df['controle'][i] = 'UNIDADES'
        
print("quantidades de unidades presentes nas mensagens:")
count_freq(df, 'Q_unidades')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'UNIDADES']])
print("unidades probs:", len(df_prob))
sample_prob('UNIDADES')

df = df[df['controle'] != 'UNIDADES']

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
#print("unidade ok:", len(df)) 
#print("unidade ok deslocado:", len(df[df['controle'] == 'DESLOCADO']))

quantidades de unidades presentes nas mensagens:


,Freq,Perc
Q_unidades,,
1,644,92.528736
2,46,6.609195
0,4,0.574713
3,2,0.287356


unidades probs: 13


# Deslocados

In [13]:
#preparando para reconhecer corretamente
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCADA','DESLOCADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DESLOCAMENTO','DESLOCADO'))

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DESLOCADO', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes > 0:
        df['controle'][i] = 'DESLOCADO'
        
count_freq(df, 'controle')
# OBS: Plantões de deslocado não são mais excluido

,Freq,Perc
controle,,
DESLOCADO,51,100.0


# (Retirada) Distancia e Saude

In [14]:
#RETIRANDO REGISTROS DE DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE OU DISPENSA
df['controle2'] = np.nan
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSADA','DISPENSADO'))
df['mensagem'] = df['mensagem'].apply(lambda x: x.replace('DISPENSA ','DISPENSADO'))

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DISPENSADO|SAUDE', mensagem)
    quantidade_observacoes = len(observacoes)

    if quantidade_observacoes >= 1:
        df['controle'][i] = 'DISP_SAUDE'
        if quantidade_observacoes == 1:
            df['controle2'][i] = observacoes[0]
        
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'DISP_SAUDE']])
print("unidades probs:", len(df_prob))
sample_prob('DISP_SAUDE')

df = df[df['controle'] != 'DISP_SAUDE']
print("unidade ok:", len(df)) 

print(df_prob['mensagem'][df_prob['controle'] == 'DISP_SAUDE'].sample(2))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,51,92.727273
DISP_SAUDE,4,7.272727


unidades probs: 17
unidade ok: 688
117      21/09 VILLA THASSIO DISPENSADO
81     21/09 OSASCO NATHÁLIA DISPENSADO
Name: mensagem, dtype: object


In [15]:
for i in range(0, len(df)):
    mensagem = df['mensagem'][i].upper()
    regex_D = re.findall(r'D[0-9]|DISTANCIA', mensagem)
    
    if len(regex_D) == 0:
        continue
    else:
        df['controle'][i] == 'DISTANCIAS'

count_freq(df, 'controle')

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'DISTANCIAS']])
print("probs:", len(df_prob))
sample_prob('DISTANCIAS')

df = df[df['controle'] != 'DISTANCIAS']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,51,100.0


probs: 17
ok: 688


# Extras

In [16]:
df["extra"] = np.nan

for i in range(0, len(df)):
    if 'EXTRA' in df['mensagem'][i].upper(): #se conter a palavra "extra"
        df['extra'][i] = 'extra' #marque na coluna extra como extra
        
print("Quantidade de registros Extra - Voluntario:")
count_freq(df, 'extra')      

Quantidade de registros Extra - Voluntario:


,Freq,Perc
extra,,
extra,4,100.0


# Tipo entrada e saida

In [17]:
#Indentificando se é um registro de entrada ou saida
df['tipo'] = np.nan

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    entradas_saidas = re.findall(r'ENTRADA|SAIDA', mensagem)
    quantidade_entradas_saidas = len(entradas_saidas)
    
    if quantidade_entradas_saidas != 1: #capturando apenas com uma informação escrita
        df['controle'][i] = 'SEM_ENTRADA_OU_SAIDA'
    else: #restante com mais de uma informação
        df['tipo'][i] = entradas_saidas[0]
        
count_freq(df, 'tipo')
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'SEM_ENTRADA_OU_SAIDA']])
print("unidades probs:", len(df_prob))
sample_prob('SEM_ENTRADA_OU_SAIDA')

df = df[df['controle'] != 'SEM_ENTRADA_OU_SAIDA']
print("unidade ok:", len(df)) 

#print(df_prob['mensagem'][df_prob['controle'] == 'SEM_ENTRADA_OU_SAIDA'].sample(1))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

,Freq,Perc
tipo,,
ENTRADA,359,52.87187
SAIDA,320,47.12813


,Freq,Perc
controle,,
DESLOCADO,43,82.692308
SEM_ENTRADA_OU_SAIDA,9,17.307692


unidades probs: 26
unidade ok: 679
ok: 679


# -----------------------------------------

# Data

In [18]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = np.nan

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio

In [19]:
#arrumando data d/mm (6/10, não 06/10)
for i in range(0, len(df)):
    if df['data'][i][1] == '/':
        df['data'][i] = '0'+df['data'][i]
df['data'].unique()

array(['20/09', '21/09', '22/09', '22/08', '23/09', '23/9', '24/09',
       '24/9'], dtype=object)

In [20]:
#somando informações restantes na data: dd/MM/yyyy
for i in range(0, len(df)):
    df['data'][i] = df['data'][i][0:2] + "/"+ mes +"/2024" 
    
count_freq(df, 'data')

,Freq,Perc
data,,
23/09/2024,222,32.695140
24/09/2024,183,26.951399
21/09/2024,162,23.858616
22/09/2024,63,9.278351
20/09/2024,49,7.216495


In [21]:
df['sem_ou_fds'] = 'semana'

for i in lista_fds:
    datinha = i+"/"+mes+"/2024"
    df['sem_ou_fds'][df['data'] == datinha] = 'fds'

count_freq(df, 'sem_ou_fds')

,Freq,Perc
sem_ou_fds,,
semana,454,66.863034
fds,225,33.136966


In [22]:
print(df.groupby(['data', 'sem_ou_fds']).size().unstack())

sem_ou_fds    fds  semana
data                     
20/09/2024    NaN    49.0
21/09/2024  162.0     NaN
22/09/2024   63.0     NaN
23/09/2024    NaN   222.0
24/09/2024    NaN   183.0


In [23]:
#df['data'].iloc[180] = '26/09/2022'

# Hora

In [24]:
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

In [25]:
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas == 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle'][i] = 'HORA'
        
count_freq(df, 'controle')

#salvando
df_prob = pd.concat([df_prob, df[df['controle'] == 'HORA']])
print("probs:", len(df_prob))
sample_prob('HORA')

df = df[df['controle'] != 'HORA']
print("ok:", len(df)) 

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
controle,,
DESLOCADO,43,95.555556
HORA,2,4.444444


probs: 28
ok: 677


In [26]:
#adicionando 0 no primeiro algorismo dos valores h:mm
for i in range(0, len(df)):
    if df['hora'][i][1] == ":":
        df['hora'][i] = "0"+df['hora'][i]
        
df['hora'].unique()

array(['20:35', '20:45', '20:50', '05:30', '19:20', '07:00', '20:55',
       '21:10', '17:35', '21:05', '09:00', '20:40', '20:10', '21:40',
       '19:50', '21:51', '21:45', '22:39', '23:40', '01:45', '03:50',
       '04:20', '04:45', '05:00', '05:15', '05:40', '05:20', '06:00',
       '05:45', '06:30', '05:50', '19:00', '07:55', '06:15', '08:20',
       '09:40', '10:00', '10:10', '10:15', '10:20', '10:30', '10:40',
       '10:35', '10:45', '10:50', '10:55', '11:20', '12:00', '12:15',
       '12:20', '12:25', '05:35', '11:45', '12:50', '12:10', '13:05',
       '13:20', '13:10', '13:30', '13:00', '13:40', '13:43', '13:50',
       '14:00', '14:20', '12:24', '14:25', '14:45', '15:20', '15:07',
       '15:50', '16:10', '16:05', '16:20', '16:30', '16:25', '16:35',
       '15:30', '16:40', '08:00', '17:25', '17:40', '18:10', '18:50',
       '19:30', '19:40', '20:00', '20:20', '21:15', '21:25', '07:20',
       '20:05', '08:30', '08:40', '11:00', '11:40', '11:50', '07:40',
       '12:40', '13:

# Hora final

In [27]:
#pacotes para manipular data
from datetime import date
from datetime import time
from datetime import datetime
from datetime import timedelta

In [28]:
df['hora'].unique()

array(['20:35', '20:45', '20:50', '05:30', '19:20', '07:00', '20:55',
       '21:10', '17:35', '21:05', '09:00', '20:40', '20:10', '21:40',
       '19:50', '21:51', '21:45', '22:39', '23:40', '01:45', '03:50',
       '04:20', '04:45', '05:00', '05:15', '05:40', '05:20', '06:00',
       '05:45', '06:30', '05:50', '19:00', '07:55', '06:15', '08:20',
       '09:40', '10:00', '10:10', '10:15', '10:20', '10:30', '10:40',
       '10:35', '10:45', '10:50', '10:55', '11:20', '12:00', '12:15',
       '12:20', '12:25', '05:35', '11:45', '12:50', '12:10', '13:05',
       '13:20', '13:10', '13:30', '13:00', '13:40', '13:43', '13:50',
       '14:00', '14:20', '12:24', '14:25', '14:45', '15:20', '15:07',
       '15:50', '16:10', '16:05', '16:20', '16:30', '16:25', '16:35',
       '15:30', '16:40', '08:00', '17:25', '17:40', '18:10', '18:50',
       '19:30', '19:40', '20:00', '20:20', '21:15', '21:25', '07:20',
       '20:05', '08:30', '08:40', '11:00', '11:40', '11:50', '07:40',
       '12:40', '13:

In [29]:
#criando uma coluna datetime para fazer calculos de horas e minutos
df['hora2'] = df['hora'].apply(lambda x: datetime.strptime(x, '%H:%M'))

#verificando se esta tudo certo
print(df['hora2'].dt.year.unique())
print(df['hora2'].dt.month.unique())
print(df['hora2'].dt.day.unique())
print(df['hora2'].dt.hour.unique())

for i in range(0, len(df)):
    minuto = int(df['hora'][i][3:])
    
    if minuto != 0 and minuto != 30:
        if minuto < 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (0) subtraindo quantos minutos tem entre o momento e HH:00
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-minuto))[11:-3]
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (30) acrecentando quantos minutos faltam para HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(30-minuto)))[11:-3]
            else:
                df['controle'][i] = 'Hora_final'
                
        elif minuto > 30:
            if df['tipo'][i] == "ENTRADA": #arredondando para baixo (30) subtraindo quando minutos tem entre o momento e HH:30
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=-(minuto-30)))[11:-3]   
            elif df['tipo'][i] == "SAIDA": #arredondando para cima (60) acrecentando quantos minutos faltam para HH:60
                df['hora'][i] = str(df['hora2'][i] + timedelta(minutes=+(60-minuto)))[11:-3]            
            else:
                df['controle'][i] = 'Hora_final'
        else:
            df['controle'][i] = 'Hora_final'
            
    elif minuto == 0 or minuto == 30:
        df['hora'][i] = str(df['hora2'][i])[11:-3] 
        
    else:
        df['controle'][i] = 'Hora_final'

df['hora'].unique()

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'Hora_final']])
print("probs:", len(df_prob))
sample_prob('Hora_final')

df = df[df['controle'] != 'Hora_final']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

[1900]
[1]
[1]
[20  5 19  7 21 17  9 22 23  1  3  4  6  8 10 11 12 13 14 15 16 18]
probs: 28
ok: 677


# Setor

In [30]:
#reconhendo em qual periodo do dia aconteceu o plantão
df['ENTRADA'] = ''
df['SAIDA'] = ''

df['PERIODO'] = ''

for i in range(0, len(df)):
    minuto = int(df['hora'][i][:2])
    
    if df['tipo'][i] == 'ENTRADA':
        df['ENTRADA'][i] = df['hora'][i]
        if minuto < 7: #periodo 6h
            df['PERIODO'][i] = '6h'
            df['SAIDA'][i] = '07:00'
        elif minuto >= 7 and minuto < 13: #periodo manha
            df['PERIODO'][i] = 'manha'
            df['SAIDA'][i] = '13:00'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            df['PERIODO'][i] = 'tarde'
            df['SAIDA'][i] = '19:00'
        elif minuto >= 19: #periodo tarde
            df['PERIODO'][i] = 'noturno entrada'
            df['SAIDA'][i] = '07:00'
        else:
            df['controle'][i] = 'ANALISAR'
    elif df['tipo'][i] == 'SAIDA':
        df['SAIDA'][i] = df['hora'][i]
        if minuto > 7 and minuto < 13: #periodo manha
            df['ENTRADA'][i] = '07:00'
            df['PERIODO'][i] = 'manha'
        elif minuto >= 13 and minuto < 19: #periodo tarde
            if df['hora'][i] == '13:00':
                df['SAIDA'][i] = '13:00'
                df['ENTRADA'][i] = '07:00'
                df['PERIODO'][i] = 'manha'
            else:
                df['ENTRADA'][i] = '13:00'
                df['PERIODO'][i] = 'tarde'
        elif minuto >= 19: #periodo noturno
            if df['SAIDA'][i] == '19:00':
                df['ENTRADA'][i] = '13:00'
                df['PERIODO'][i] = 'tarde'
            else:
                df['ENTRADA'][i] = '19:00'
                df['PERIODO'][i] = 'noturno saida'
        elif minuto <= 7: #periodo noturno entrada
            df['ENTRADA'][i] == '19:00'
            df['PERIODO'][i] = 'noturno saida'
        else:
            df['controle'][i] = 'ANALISAR'
    else:
        df['controle'][i] = 'HORARIOS'

df['controle'][df['ENTRADA'] == df['SAIDA']] = 'HORARIOS'

count_freq(df, 'PERIODO')
count_freq(df, 'controle')

df['periodo'] = np.nan
df['periodo'][df['PERIODO'] == '6h'] = 1
df['periodo'][df['PERIODO'] == 'manha'] = 2
df['periodo'][df['PERIODO'] == 'tarde'] = 3
df['periodo'][df['PERIODO'] == 'noturno saida'] = 4
df['periodo'][df['PERIODO'] == 'noturno entrada'] = 5

#salvando restante
df_prob = pd.concat([df_prob, df[df['controle'] == 'HORARIOS']])
print("probs:", len(df_prob))
sample_prob('HORARIOS')

df = df[df['controle'] != 'HORARIOS']
print("ok:", len(df))

df_prob.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)

,Freq,Perc
PERIODO,,
manha,293,43.279173
tarde,173,25.553914
6h,120,17.725258
noturno saida,89,13.146233
noturno entrada,2,0.295421


,Freq,Perc
controle,,
DESLOCADO,43,100.0


probs: 28
ok: 677


# Plantões noturnos

In [31]:
from datetime import timedelta

filtro = df['periodo'] == 5

# Supondo que df[filtro] já esteja definido
for i in df[filtro].index:
    # Imprime a data original
    print(df['data'][i])
    
    # Converte a data para datetime, subtrai um dia e converte de volta para string
    data_atualizada = (pd.to_datetime(df['data'][i], format='%d/%m/%Y') - timedelta(days=1)).strftime('%d/%m/%Y')
    
    # Imprime a data atualizada
    df['data'][i] = data_atualizada

20/09/2024
21/09/2024


# Duração do Plantão

df['DURAÇÃO'] = ''

for i in range(0, len(df)):
    diferenca_tempo = datetime.strptime(df['SAIDA'][i], '%H:%M') - datetime.strptime(df['ENTRADA'][i], '%H:%M')
    
    #diferenca_tempo vem em formato estranho, tem-se que transformar em segundos antes
    diferenca_tempo_s = diferenca_tempo.seconds
    horas = '0' + str(int((diferenca_tempo_s/60)/60)) #capturando horas
    minutos = str(int((diferenca_tempo_s/60)%60))  #capturando minutos
    
    if len(minutos) == 1:
        minutos = '0' + minutos
    
    df['DURAÇÃO'][i] = horas + ':' + minutos
    
df['DURAÇÃO'][df['DURAÇÃO'] == '00:00'] = '06:00'    

df['DURAÇÃO'].unique()

# Checagem Unidades:

In [32]:
# alterando JABA e AVVICENA
def replace_values(value):
    if value == 'JABA':
        return 'JABAQUARA'
    elif value == 'AVICCENA':
        return 'CENTRAL DO TATUAPE'
    elif value == 'SANTA':
        return 'SANTA ISABEL'
    else:
        return value
    
def replace_values2(value):
    if isinstance(value, list):
        return [replace_values(v) for v in value]
    else:
        return replace_values(value)

# Aplicar a função à coluna 'setor'
df['unidade'] = df['unidade'].apply(lambda x: replace_values2(x))

In [33]:
# Cria uma nova Série sem as listas
serie_sem_listas = pd.Series([x for x in df['unidade'] if not isinstance(x, list)])

# Imprime os valores únicos da nova Série
print(serie_sem_listas.unique())

['ANALIA' 'OSASCO' 'ITAIM' 'MORUMBI' 'JABAQUARA' 'VILLA'
 'CENTRAL DO TATUAPE' 'SANTA ISABEL' 'VNS' 'MATERNIDADE' 'ALPHA']


In [34]:
# Cria uma lista com apenas os valores que são listas
listas = [x for x in df['unidade'] if isinstance(x, list)]

# Converte as listas em tuplas e adiciona elas a um conjunto para obter apenas as listas distintas
listas_distintas = set(tuple(lista) for lista in listas)

# Imprime as listas distintas
for lista in listas_distintas:
    print(list(lista))


['OSASCO', 'VILA']
['ANALIA', 'MATERNIDADE']
['OSASCO', 'ITAIM']
['MORUMBI', 'VNS']
['ANALIA', 'ITAIM']
['ANALIA', 'OSASCO']
['VILLA', 'ITAIM']
['ALPHA', 'ITAIM']
['JABAQUARA', 'VILLA']
['ITAIM', 'MATERNIDADE']
['MORUMBI', 'ANALIA']
['JABAQUARA', 'VNS']
['CENTRAL DO TATUAPE', 'ITAIM']
['VNS', 'VNS']
['MORUMBI', 'JABAQUARA']
['OSASCO', 'ALPHA']
['MORUMBI', 'VILA', 'ITAIM']
['MORUMBI', 'OSASCO']
['SANTA ISABEL', 'ANALIA']
['ANALIA', 'VNS']
['VILLA', 'VNS']
['VNS', 'ITAIM']


# Setor (2)

In [35]:
df['setor'] = np.nan

for i in range(0, len(df)):
    if df['sem_ou_fds'][i] == 'semana':
        if df['PERIODO'][i] == '6h':
            #df['setor'][i] = 'SEMANA - 6HS'
            df['setor'][i] = '01 - NOTURNA - ENTRADA ANTES DAS 07HS'
        elif df['PERIODO'][i] == 'manha':
            #df['setor'][i] = 'SEMANA - MANHA - C.CIRURGICO'
            df['setor'][i] = '04 - SEMANA - CENTRO CIRURGICO'
        elif df['PERIODO'][i] == 'tarde':
            #df['setor'][i] = 'SEMANA - TARDE - C.CIRURGICO'
            df['setor'][i] = '04 - SEMANA - CENTRO CIRURGICO'
        elif df['PERIODO'][i] == 'noturno saida':
            #df['setor'][i] = 'SEMANA - NOTURNO - SAIDA'
            df['setor'][i] = '07 - NOTURNA - SAIDA APOS 19HS'
        elif df['PERIODO'][i] == 'noturno entrada':
            df['setor'][i] = '09 - NOTURNA - PLANTAO'
        else:
            df['setor'][i] = 'VERIFICAR'

    elif df['sem_ou_fds'][i] == 'fds': 
        if df['PERIODO'][i] == '6h':
            #df['setor'][i] = 'FDS - 6HS'
            df['setor'][i] = '11 - FDS - 06 HS'
        elif df['PERIODO'][i] == 'manha':
            #df['setor'][i] = 'FDS - C.CIRURGICO ROTINA'
            df['setor'][i] = '13 - FDS - CENTRO CIRURGICO'
        elif df['PERIODO'][i] == 'tarde':
            #df['setor'][i] = 'FDS - SAIDA'
            df['setor'][i] = '13 - FDS - CENTRO CIRURGICO'
        elif df['PERIODO'][i] == 'noturno saida':
            #df['setor'][i] = 'FDS - PLANTAO'
            df['setor'][i] = '14 - FDS - SAIDA'
        elif df['PERIODO'][i] == 'noturno entrada':
            df['setor'][i] = '20 - FDS - PLANTAO'
        else:
            df['setor'][i] = 'VERIFICAR'  

    else:
        df['setor'][i] = 'VERIFICAR'
        
df['setor'] = df['setor'].apply(lambda x: unidecode.unidecode(x.replace('  ',' ').upper().strip()))
count_freq(df, 'setor')

,Freq,Perc
setor,,
04 - SEMANA - CENTRO CIRURGICO,294,43.426883
13 - FDS - CENTRO CIRURGICO,172,25.406204
01 - NOTURNA - ENTRADA ANTES DAS 07HS,81,11.964549
07 - NOTURNA - SAIDA APOS 19HS,77,11.373708
11 - FDS - 06 HS,39,5.760709
14 - FDS - SAIDA,12,1.772526
09 - NOTURNA - PLANTAO,1,0.147710
20 - FDS - PLANTAO,1,0.147710


In [36]:
#################################################
print(df['setor'].unique())
df['setor_inicio'] = np.nan
for i in range(len(df)):
    corte = df['setor'][i].find('-')
    texto = df['setor'][i][:corte].strip()
    df['setor_inicio'][i] = texto

print(df['setor_inicio'].unique())
#################################################

['07 - NOTURNA - SAIDA APOS 19HS' '01 - NOTURNA - ENTRADA ANTES DAS 07HS'
 '04 - SEMANA - CENTRO CIRURGICO' '11 - FDS - 06 HS'
 '13 - FDS - CENTRO CIRURGICO' '14 - FDS - SAIDA' '09 - NOTURNA - PLANTAO'
 '20 - FDS - PLANTAO']
['07' '01' '04' '11' '13' '14' '09' '20']


# *****************************************

# Pareamento

In [37]:
print(len(df))
print(len(df_prob))

677
28


In [38]:
df_prob_pareado = pd.DataFrame(columns=df.columns)
df_prob_pareado

,controle,chat,dia,nome,mensagem,data,unidade,Q_unidades,controle2,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO,periodo,setor,setor_inicio


In [39]:
count_ok = 0
count_erro = 0

for nome in sorted(df['nome'].unique()):
    df_filter = df[df['nome'] == nome]
    for data in sorted(df_filter['data'].unique()):
        df_filter = df[(df['nome'] == nome) & (df['data'] == data)]

        if len(df_filter) >= 2:
            qtd_entrada = len(df_filter[df_filter['tipo'] == 'ENTRADA'])
            qtd_saida = len(df_filter[df_filter['tipo'] == 'SAIDA'])

            if qtd_entrada == qtd_saida:
                print(nome, '|', data)
                count_ok += 1
        else:
            # Procurar se o registro foi mandado com o erro:
            df_prob_filter = df_prob[(df_prob['nome'] == nome) & (df_prob['data'] == data)]

            # ERRO PARCIAL
            if len(df_prob_filter) > 0: 
                print(nome, '|', data, '|', 'PARCIAL')
                df_prob['controle'][df_prob_filter.index] = 'erro_parcial'
                
            # ERRO DE PERDA TOTAL
            else: 
                print(nome, '|', data, '|', 'ERRO')
                df_filter['controle'] = 'erro_total'
                df_prob_pareado = pd.concat([df_prob_pareado, df_filter])

            count_erro += 1
    print('='*50)

ALEXANDRE BERNARDINO | 23/09/2024 | ERRO


ALEXANDRE BERNARDINO | 24/09/2024 | ERRO
ALEXSANDER TAKASHI HAMADA | 21/09/2024
ALEXSANDER TAKASHI HAMADA | 22/09/2024
ALEXSANDER TAKASHI HAMADA | 23/09/2024
ALEXSANDER TAKASHI HAMADA | 24/09/2024 | ERRO
ALINE FRANCHINI GOULART LIMA | 21/09/2024
ALINE FRANCHINI GOULART LIMA | 24/09/2024
AMANDA KIYOKO SOUZA KOSSAKA | 20/09/2024
AMANDA KIYOKO SOUZA KOSSAKA | 24/09/2024
ANA JUNG DOCKO | 21/09/2024
ANA JUNG DOCKO | 22/09/2024
ANA JUNG DOCKO | 23/09/2024
ANDRE AUGUSTO BELLINI | 22/09/2024
ANDRE AUGUSTO BELLINI | 24/09/2024
ANDRE DA COSTA AGUIAR VENTURA | 21/09/2024
ANDRE DA COSTA AGUIAR VENTURA | 22/09/2024
ANDRE DA COSTA AGUIAR VENTURA | 24/09/2024
ANDREIA LOUREIRO MORI | 23/09/2024
ANNA BEATRIZ ARANHA TREVISAN | 20/09/2024
ANNA BEATRIZ ARANHA TREVISAN | 23/09/2024
ANTONIO ALVARO DIAS LIMA DA CUNHA NETO | 21/09/2024
ARTHUR OLIVEIRA FACCHINI | 21/09/2024
ARTHUR OLIVEIRA FACCHINI | 24/09/2024 | ERRO
ARTHUR RODRIGUES TORRELIO | 21/09/2024 | ERRO
ARTHUR RODRIGUES TORRELIO | 24/09/2024
AUGUSTO 

# (0) Salvando problemas de entrada e saida

In [40]:
print(count_ok)
print(count_erro)

275
111


In [41]:
salvando_nome_arquivo('df_entrada_saida.xlsx', df_prob_pareado)

92  +=  109 => 201


# Ultimos ajustes:

In [42]:
df = df[['chat', 'dia', 'nome', 'mensagem', 'controle', 'data', 'unidade',
       'extra', 'tipo', 'ENTRADA', 'SAIDA', #'DURAÇÃO', 
       'PERIODO','periodo',
         'setor', 'sem_ou_fds']]

# =========================================

# Deslocados:

In [43]:
df['controle'][(df['PERIODO'] == '6h') & (df['controle'] == 'DESLOCADO')] = 'DESLOCADO 6H'
count_freq(df, 'controle')

,Freq,Perc
controle,,
DESLOCADO,32,74.418605
DESLOCADO 6H,11,25.581395


In [44]:
df['unidade2'] = df['unidade']

for i in range(0, len(df)):
    if isinstance(df.loc[i, 'unidade'], list):
        df['unidade'][i] = df['unidade'][i][0]

df['unidade'][df['unidade'] == 'VILA'] = 'VILLA'
df['unidade'][df['unidade'] == 'JABA'] = 'JABAQUARA'
df['unidade'][df['unidade'] == 'SANTA'] = 'SANTA ISABEL'

sorted(df['unidade'].unique())
count_freq(df, 'unidade')

,Freq,Perc
unidade,,
ANALIA,124,18.316100
JABAQUARA,106,15.657312
MORUMBI,101,14.918759
ITAIM,85,12.555391
OSASCO,84,12.407681
VILLA,70,10.339734
SANTA ISABEL,44,6.499261
VNS,34,5.022157
MATERNIDADE,13,1.920236


# (1) Salvando tudo: gabs & giov

In [45]:
salvando_nome_arquivo('df_gabs.xlsx', df)
print('.')
salvando_nome_arquivo('df_giov.xlsx', df_prob)

677
.
136  +=  28 => 164


# =========================================

# Dispensados:

In [46]:
df = pd.read_excel('df_giov.xlsx')[['chat','dia','nome','mensagem','controle']]
df = df.drop_duplicates(subset=['chat'])
df = df.sort_values(['nome', 'dia'])
print(len(df))
df.sample(3)

164


,chat,dia,nome,mensagem,controle
130,[14/09/2024 07:56:41] Silvia Aline dos Santos ...,14/09/2024,SILVIA ALINE DOS SANTOS ANDRADE,14/09 JABAQUARA - SILVIA ENTRADA 08:00,NaN
129,[20/09/2024 06:41:30] Sheila Itiroco Monte: 20...,20/09/2024,SHEILA ITIROCO MONTE,20/09 JABAQUARA SHEILA ENTRADA 07:00,NaN
44,[11/09/2024 16:48:27] BackOffice Pacientes: MA...,11/09/2024,BACKOFFICE PACIENTES,MAURÍCIO FARIA TETTI SAÍDA - DATA DE SAÍDA:...,SEM_ENTRADA_OU_SAIDA


In [47]:
count_freq(df, 'controle')

,Freq,Perc
controle,,
SEM_ENTRADA_OU_SAIDA,82,56.164384
UNIDADES,25,17.123288
DATA_DO_PLANTÃO,24,16.438356
DISP_SAUDE,8,5.479452
HORA,5,3.424658
erro_parcial,2,1.369863


In [48]:
#RETIRANDO REGISTROS DE DISTANCIA E SAIDAS POR QUESTÃO DE SAUDE OU DISPENSA

df['controle2'] = np.nan

#preparando para reconhecer corretamente

for i in range(0, len(df)):
    mensagem = unidecode.unidecode(df['mensagem'][i].upper())
    observacoes = re.findall(r'DISPENSADO|SAUDE|DISPENSADA', mensagem)
    quantidade_observacoes = len(observacoes)
    
    if quantidade_observacoes == 0:
        df['controle2'][i] = 'PROB'
        
df['controle2'].value_counts() 

#salvando
df_prob = df[df['controle2'] == 'PROB']
df_prob.reset_index(drop=True, inplace=True)
print("unidades probs:", len(df_prob))

df = df[df['controle2'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("unidade ok:", len(df)) 

unidades probs: 156
unidade ok: 8


In [49]:
#DATA DO PLANTÃO
df['controle2'] = ''
df['data'] = ''

for i in range(0, len(df)):
    lista_datas = re.findall(r'[0-9]{1,2}\/[0-9]{1,2}', df['mensagem'][i]) #regex dd/mm
    quantidade_itens = len(lista_datas)
    
    if quantidade_itens == 1: #capturar apenas registros com uma data apenas
        df['data'][i] = lista_datas[0]
        df['controle2'][i] = 'NORM'
    else: #restante com nenhuma ou mais de uma data
        df['controle2'][i] = 'PROB'

#salvando
df_prob = pd.concat([df_prob, df[df['controle2'] != 'NORM']])
df_prob.reset_index(drop=True, inplace=True)
print("data probs:", len(df_prob))

df = df[df['controle2'] != 'PROB']
df.reset_index(drop=True, inplace=True)
print("datas ok:", len(df)) 

df['data'] = df['data']+"/2024"
sorted(df['data'].unique())

data probs: 156
datas ok: 8


['07/09/2024', '09/09/2024', '19/09/2024', '20/09/2024', '21/09/2024']

In [50]:
#retirando a data da mensagem para não atrapalhar no reconhecimento de hora
df['hora'] = ''

for i in range(0, len(df)):
    df['hora'][i] = df['mensagem'][i].replace(df['data'][i], '').strip() #substituindo a data por vazio
    
#substituindo "h" por ":" para capturar horarios 11h22
df['hora'] = df['hora'].apply(lambda x: x.upper().replace("H", ":"))

#Retirando aqueles que contem horas no registro da mensagem
for i in range(0, len(df)):
    mensagem = df['hora'][i]
    horas = re.findall(r'[0-9]{1,2}:[0-9]{1,2}', mensagem) #capturando horas HH:mm
    quantidade_horas = len(horas)
    
    if quantidade_horas >= 1: #capturando apenas com uma horas escrita
        df['hora'][i] = horas[0]
        df['controle2'][i] = 'PROB'
    else: #restante com nenhuma ou mais horas (geralmente pq digitou a hora de entrada e saida {informação inutil})
        df['hora'][i] = ''
        df['controle2'][i] = 'OK'
        
print(df['controle2'].value_counts())

#salvando
df_prob = pd.concat([df_prob, df[df['controle2'] != 'OK']])
df_prob.reset_index(drop=True, inplace=True)
print("probs:", len(df_prob))

df = df[df['controle2'] == 'OK']
df.reset_index(drop=True, inplace=True)
print("ok:", len(df)) 

controle2
PROB    4
OK      4
Name: count, dtype: int64
probs: 160
ok: 4


In [51]:
print(len(df_prob))
print(len(df))

160
4


# (2) Salvando tudo: giov & dispensados

In [52]:
df = df.drop_duplicates(subset=['chat'])

os.remove("df_giov.xlsx")
salvando_nome_arquivo('df_giov.xlsx', df_prob)
print('.')
salvando_nome_arquivo('df_dispensados.xlsx', df)

160
.
4


# ===========================================

# Ultima Contagem

In [53]:
print(len(pd.DataFrame({'chat': read_file('_chat.txt') })))
print('.')
print(len(pd.read_excel('df_dispensados.xlsx')))
print('.')
print(len(pd.read_excel('df_gabs.xlsx')))
print('.')
print(len(pd.read_excel('df_giov.xlsx')))

705
.


4
.
677
.
160


# ==================================

In [54]:
stop

NameError: name 'stop' is not defined

In [ ]:
print(len(pd.read_excel('df_dispensados.xlsx')))
print('.')
print(len(pd.read_excel('df_gabs.xlsx')))
print('.')
print(len(pd.read_excel('df_giov.xlsx')))

In [ ]:
df1 = pd.read_excel('df_dispensados.xlsx')[['chat','nome','mensagem']]
os.remove('df_dispensados.xlsx')
df2 = pd.read_excel('df_gabs.xlsx')[['chat','nome','mensagem']]
os.remove('df_gabs.xlsx')
df3 = pd.read_excel('df_giov.xlsx')[['chat','nome','mensagem']]
os.remove('df_giov.xlsx')

df = pd.concat([df1, df2, df3]).reset_index(drop=True)
len(df)

In [ ]:
fd = pd.read_excel('Feitos.xlsx')[['chat','nome','mensagem']]
len(fd)

In [ ]:
df_residuos = pd.DataFrame(columns=['chat','nome','mensagem'])

count = 0
for i in range(len(df)):
    filter = fd[fd['chat'] == df['chat'][i]]

    if len(filter) == 1:
        pass
    elif len(filter) > 1:
        print('duplicada')
    elif len(filter) == 0:
        count += 1
        df_residuos = pd.concat([df_residuos, df.iloc[[i]] ])


print(count)
df_residuos

In [ ]:
df_residuos.to_excel('df_residuos.xlsx')

In [ ]:
total = len(fd) + len(df_residuos)

percent = len(fd) / total
percent = round(percent*100, 2)
percent = str(percent) + "%"
percent = "("+percent+")"
print("Registros App:", len(fd), percent)

percent = len(df_residuos) / total
percent = round(percent*100, 2)
percent = str(percent) + "%"
percent = "("+percent+")"
print("Registros Giovanelli", len(df_residuos), percent)
print("#"*30, "\n")

for nome in sorted(list(df_residuos['nome'].unique())):
    print(nome, "="*10)
    selecao = df_residuos.loc[df_residuos['nome']==nome]
    selecao.reset_index(drop=True, inplace=True)
    
    for i in range(len(selecao)):
        print(selecao["mensagem"][i])
    
    print("\n")